This is the all-in-one notebook of both train.py and infer.py along with all preprocessing functions for the data. Currently this loads data from the shared Google drive which is why the drive must be mounted and then the file paths all consist of '/content/drive/My Drive/EHR_Dream_Challenge/'. 

However, if downloading as a zip file to run end-to-end on local machine, please remove '/content/drive/My Drive/EHR_Dream_Challenge/' from all filepaths


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
assert tf.__version__ >= "2.0"
print(tf.__version__)


2.1.0-rc1


In [0]:
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import copy
import pickle
from tqdm import tqdm
from tqdm import tqdm_notebook
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import OrderedDict

In [0]:
test = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/train/person.csv')
test.head(20)

,year_of_birth,gender_source_value,ethnicity_concept_id,provider_id,race_source_concept_id,person_id,person_source_value,month_of_birth,gender_source_concept_id,ethnicity_source_concept_id,care_site_id,day_of_birth,ethnicity_source_value,location_id,race_concept_id,gender_concept_id,birth_datetime,race_source_value
0,1937,2,38003564,NaN,NaN,19076,3FE03A5F3E571720,8,NaN,NaN,NaN,1,1,404,8557.0,8532,1937-08-01,1
1,1971,2,38003564,NaN,NaN,63615,D6EA175DFA84014F,12,NaN,NaN,NaN,1,1,59,NaN,8532,1971-12-01,1
2,1935,2,38003564,NaN,NaN,138812,D49E41F7F3EC3403,9,NaN,NaN,NaN,1,1,464,8516.0,8532,1935-09-01,1
3,1943,1,38003564,NaN,NaN,24089,50B972E03C6FDBAE,12,NaN,NaN,NaN,1,1,345,8515.0,8507,1943-12-01,1
4,1933,2,38003564,NaN,NaN,21093,46A9A1FFC0CCBF8D,7,NaN,NaN,NaN,1,1,363,NaN,8532,1933-07-01,1
5,1935,1,38003564,NaN,NaN,75082,FD5E3EAD6D1BDEC0,1,NaN,NaN,NaN,1,2,1689,8657.0,8507,1935-01-01,2
6,1940,1,38003564,NaN,NaN,84517,1C7C0F2B5F661A86,5,NaN,NaN,NaN,1,1,495,NaN,8507,1940-05-01,1
7,1936,1,38003564,NaN,NaN,57878,C37DDF89936F0DF5,4,NaN,NaN,NaN,1,1,864,8527.0,8507,1936-04-01,1
8,1927,1,38003564,NaN,NaN,58192,C48EE494A10C1C70,4,NaN,NaN,NaN,1,1,208,8527.0,8507,1927-04-01,1
9,1932,2,38003564,NaN,NaN,21703,48B0EDE6178E24B9,3,NaN,NaN,NaN,1,1,618,8657.0,8532,1932-03-01,1


In [0]:
test2 = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/train/condition_occurrence.csv')
test2.head(20)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,provider_id,visit_detail_id,condition_start_datetime,condition_end_datetime,condition_source_concept_id,person_id,condition_status_source_value,condition_end_date,condition_start_date,condition_status_concept_id,condition_type_concept_id,condition_concept_id,stop_reason,condition_source_value,condition_occurrence_id,visit_occurrence_id
0,461776.0,NaN,2008-11-20,2008-11-20,44829639.0,115208.0,NaN,2008-11-20,2008-11-20,4230359.0,38000230.0,312922.0,NaN,V4581,9070047.0,2864116
1,275286.0,NaN,2009-02-18,2009-02-22,44825447.0,115208.0,NaN,2009-02-22,2009-02-18,4230359.0,38000230.0,43020432.0,NaN,44021,5258637.0,3105196
2,617439.0,NaN,2008-08-24,2008-08-24,44828074.0,115208.0,NaN,2008-08-24,2008-08-24,NaN,38000230.0,75354.0,NaN,7329,12839271.0,4169690
3,1418.0,NaN,2010-05-14,2010-05-14,44830080.0,115208.0,NaN,2010-05-14,2010-05-14,4230359.0,38000230.0,42872402.0,NaN,41401,4019921.0,2042432
4,28408.0,NaN,2009-02-08,2009-02-08,44828981.0,115208.0,NaN,2009-02-08,2009-02-08,4230359.0,38000230.0,316999.0,NaN,4279,4022174.0,1329697
5,88207.0,NaN,2009-03-16,2009-03-16,44823450.0,115208.0,NaN,2009-03-16,2009-03-16,4230359.0,38000230.0,77670.0,NaN,78659,2025129.0,2856939
6,257875.0,NaN,2009-10-04,2009-10-04,44823445.0,115208.0,NaN,2009-10-04,2009-10-04,NaN,38000230.0,439926.0,NaN,78079,2489912.0,2703344
7,18869.0,NaN,2010-02-02,2010-02-02,44836342.0,115208.0,NaN,2010-02-02,2010-02-02,NaN,38000230.0,75373.0,NaN,81315,2036349.0,1522254
8,86984.0,NaN,2009-11-05,2009-11-05,44830325.0,89460.0,NaN,2009-11-05,2009-11-05,4230359.0,38000230.0,72993.0,NaN,71516,1571566.0,49511
9,296556.0,NaN,2010-01-28,2010-01-28,44825877.0,89460.0,NaN,2010-01-28,2010-01-28,4230359.0,38000230.0,434500.0,NaN,8208,14295173.0,5420353


In [0]:
def preprocess_LVED_deaths(train=True):
    #useful_cols = pd.read_csv('OMOP_usefule_columns.csv')
    if train:
        folder_fp = '/content/drive/My Drive/EHR_Dream_Challenge/train/'
        death = pd.read_csv(folder_fp + 'death.csv', usecols = ['person_id',
            'death_date', 'death_type_concept_id'])
        fp = '/content/drive/My Drive/EHR_Dream_Challenge/train/visit_occurrence.csv'
        save_append = ''
    else:
        folder_fp = '/content/drive/My Drive/EHR_Dream_Challenge/infer/'
        fp = '/content/drive/My Drive/EHR_Dream_Challenge/infer/visit_occurrence.csv'
        save_append = '_test'

    person_col_list= ['person_id','gender_concept_id',
    'year_of_birth','month_of_birth',
    'day_of_birth', 'birth_datetime',
    'race_concept_id', 'ethnicity_concept_id']

    persons = pd.read_csv(folder_fp + 'person.csv', usecols = person_col_list)

    # Only in train do we have this
    if train:
        death_ids = list(death['person_id'])
        persons['died'] = persons['person_id'].isin(death_ids)

    pID_LVED_dict = queryLastVisitEndDate(fp)
    person_id_list = persons['person_id'].tolist()

    # For the synpuf, I'm filling using 1970s as dummy
    fill_val = datetime(1970, 1, 1, 0, 0)
    LVED_list = []
    for person_id in person_id_list:
        try:
            LVED_list.append(pID_LVED_dict[person_id])
        except KeyError:
            LVED_list.append(fill_val)

    persons['last_visit_end_date'] = LVED_list

    if train:
        # Save in persons table a column of "label"
        # 0 - default if person did not die
        # 0 - if person died, but after 6 months from last visit
        # 1 - if person died between 0 and 6 months after last visit

        persons['label'] = 0;
        # Get all spots in dataframe where last_visit_end_date is not empty, and the person died
        for person_id in tqdm(persons[(~persons['last_visit_end_date'].isnull()) & (persons['died'])]['person_id']):
            # Lookup the death date from death
            death_date = datetime.strptime(death[death['person_id'] == person_id]['death_date'].values[0], '%Y-%m-%d')

            # Last visit end date + 6 months
            # Something happened with typecasting...
            lved_6m = pd.to_datetime(persons[persons.person_id == person_id]['last_visit_end_date'].values[0]) + relativedelta(months=+6)

            # Check if the death_date is less than or equal to the last visit + 6 months
            if death_date <= lved_6m:
                persons.loc[(persons['person_id'] == person_id),'label'] = 1

    # Let's also find their age at LVED (birth day - LVED) and add that as a column
    birthdays = []
    age = []
    for ind, row in persons.iterrows():
        # print(ind, row.person_id)
        # print(row.year_of_birth, row.month_of_birth, row.day_of_birth)
        try:
            birthday_str = str(int(row.year_of_birth)) + '-' + str(int(row.month_of_birth)) + '-' + str(int(row.day_of_birth))
            birthday = datetime.strptime(birthday_str, '%Y-%m-%d')
            birthdays.append(birthday)
            tdelta = row.last_visit_end_date - birthday
            age.append(tdelta.days)
        except ValueError:
            # nans? What to do...
            birthdays.append(fill_val)
            age.append(0)

    persons['birthday'] = birthdays
    persons['age'] = age

    age_days = persons['age'].to_numpy()
    np.save('/content/drive/My Drive/EHR_Dream_Challenge/scratch/age_days' + save_append + '.npy', age_days)

    # Now, we save a copy of this persons_table, and the LVED dictionary.
    persons.to_csv('/content/drive/My Drive/EHR_Dream_Challenge/scratch/persons_LVED' + save_append + '.csv', index=False)

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/pID_LVED_dict' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(pID_LVED_dict, handle, pickle.HIGHEST_PROTOCOL)

    if train:
        Y = persons['label'].to_numpy()
        np.save('/content/drive/My Drive/EHR_Dream_Challenge/scratch/Y.npy', Y)

def queryLastVisitEndDate(fp):
    # Let's get just the indices of the useful rows
    useful_cols = ['visit_occurrence_id','person_id','visit_concept_id',
            'visit_start_date','visit_end_date','visit_type_concept_id']

    # let's chunk with 100000 at a time
    # wc -l 3353654
    visit_occurrence_chunker = pd.read_csv(fp, chunksize=100000, usecols=useful_cols)

    # person_id to last_visit_end_date dict
    pID_LVED_dict = OrderedDict()

    for chunk in tqdm(visit_occurrence_chunker):
	# Let's drop duplicates and NaNs
        chunk.drop_duplicates(inplace=True)
        chunk.dropna(axis=0, subset=['visit_end_date'], inplace=True)

        # Get set of all person_ids in this chunk
        person_ids_chunk = set(chunk.person_id)

        # For each person_id, get a list of last_visit_end_dates
        for person_id in person_ids_chunk:
            # Chunk of dataframe for this person_id
            tmp = chunk[chunk['person_id'] == person_id]
            # Get all visit_end_dates and find the max (last date)
            last_visit_end_date = max([datetime.strptime(x, '%Y-%m-%d') for x in tmp.visit_end_date.tolist()])
            # if key exists check if greater than existing last date, if so reset
            if person_id in pID_LVED_dict:
                if last_visit_end_date > pID_LVED_dict[person_id]:
                    pID_LVED_dict[person_id] = last_visit_end_date;
            else:
                pID_LVED_dict[person_id] = last_visit_end_date

            # persons.loc[(persons['person_id'] == person_id),'last_visit_end_date'] = last_visit_end_date

    return pID_LVED_dict

# # Get the useful rows for a given filename w/o .csv
# def queryUsefulRows(filename):
#     '''
#     Return the useful rows given the OMOP_usefule_columns.csv has already been loaded into pandas.
#     Input:
#         filename - str, name of input file (e.g. 'person' or 'death'
#     Output:
#         useful_rows - list, ['col1', 'col2']
#     '''
#     return list(useful_cols[(useful_cols['TabNam'] == filename) & (useful_cols['Useful'] == True)].ColNam)


# Functions for data processing, to make things neater
def createMatrix(person_id_to_ind_dict, concept_id_to_ind_dict, pID_concepts_dict):
    # Create numpy array of conditions for all patients.
    # e.g. x-axis: person_id, y_axis: 0-7765 (see ind_to_condition_concept_id_dict)
    data_mat = np.zeros((len(person_id_to_ind_dict), len(concept_id_to_ind_dict)))
    for person_id in tqdm(pID_concepts_dict):
        # [row = person_id, column = all concept_id_cols]
        data_mat[person_id_to_ind_dict[person_id],
                       [concept_id_to_ind_dict[x[0]] for x in pID_concepts_dict[person_id]]] += 1
    return data_mat

def make_cond_proc_drug(train=True):
    if train:
        folder_fp = '/content/drive/My Drive/EHR_Dream_Challenge/train/'
        save_append = ''
    else:
        folder_fp = '/content/drive/My Drive/EHR_Dream_Challenge/infer/'
        save_append = '_test'

    print('Folder for data loaded: ' + folder_fp)

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/pID_LVED_dict' + save_append + '.pickle', 'rb') as handle:
        pID_LVED_dict = pickle.load(handle)

    conds_dict, conds_concept_ids, conds_to_ind_dict, ind_to_conds_dict, conds_person_id_to_ind_dict = run_data_pipeline(
        folder_fp + 'condition_occurrence.csv', ['condition_concept_id', 'condition_start_date'],
        '/content/drive/My Drive/EHR_Dream_Challenge/app/condition_occurrence_concepts.csv', pID_LVED_dict, train)

    procs_dict, procs_concept_ids, procs_to_ind_dict, ind_to_procs_dict, procs_person_id_to_ind_dict = run_data_pipeline(
        folder_fp + 'procedure_occurrence.csv', ['procedure_concept_id', 'procedure_date'],
        '/content/drive/My Drive/EHR_Dream_Challenge/app/procedure_occurrence_concepts.csv', pID_LVED_dict, train)

    drugs_dict, drugs_concept_ids, drugs_to_ind_dict, ind_to_drugs_dict, drugs_person_id_to_ind_dict = run_data_pipeline(
        folder_fp + 'drug_exposure.csv', ['drug_concept_id', 'drug_exposure_start_date'],
        '/content/drive/My Drive/EHR_Dream_Challenge/app/drug_exposure_concepts.csv', pID_LVED_dict, train)

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_dict' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(conds_dict, handle, pickle.HIGHEST_PROTOCOL)

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_dict' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(procs_dict, handle, pickle.HIGHEST_PROTOCOL)

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_dict' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(drugs_dict, handle, pickle.HIGHEST_PROTOCOL)

    conds_lookups = {}
    conds_lookups['concept_ids'] = conds_concept_ids
    conds_lookups['concept_id_to_ind'] = conds_to_ind_dict
    conds_lookups['ind_to_concept_id'] = ind_to_conds_dict
    conds_lookups['person_id_to_ind'] = conds_person_id_to_ind_dict

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_lookups' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(conds_lookups, handle, pickle.HIGHEST_PROTOCOL)

    procs_lookups = {}
    procs_lookups['concept_ids'] = procs_concept_ids
    procs_lookups['concept_id_to_ind'] = procs_to_ind_dict
    procs_lookups['ind_to_concept_id'] = ind_to_procs_dict
    procs_lookups['person_id_to_ind'] = procs_person_id_to_ind_dict

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_lookups' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(procs_lookups, handle, pickle.HIGHEST_PROTOCOL)

    drugs_lookups = {}
    drugs_lookups['concept_ids'] = drugs_concept_ids
    drugs_lookups['concept_id_to_ind'] = drugs_to_ind_dict
    drugs_lookups['ind_to_concept_id'] = ind_to_drugs_dict
    drugs_lookups['person_id_to_ind'] = drugs_person_id_to_ind_dict

    with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_lookups' + save_append + '.pickle', 'wb') as handle:
        pickle.dump(drugs_lookups, handle, pickle.HIGHEST_PROTOCOL)

def run_data_pipeline(fp, column_names, standard_lookup_file, pID_LVED_dict, train):
    print('Querying timelines...')
    pID_concepts_dict = queryTimelinesFromCSV(fp, column_names)
    print('Generating all concepts...')
    all_concept_ids, concept_id_to_ind_dict, ind_to_concept_id_dict = queryAllConcepts(
        standard_lookup_file, column_names[0])
    print('Creating person ID lookup')
    person_id_to_ind_dict, ind_to_person_id_dict = createPersonToInd(train)
    print('Using cutoffs to eliminate extraneous data...')
    final_pID_concepts_dict = removeEventsWithCutoff(pID_concepts_dict,
        pID_LVED_dict, concept_id_to_ind_dict, cutoff=6)
    # print('Creating numpy data matrix...')
    # data_mat = createMatrix(person_id_to_ind_dict, concept_id_to_ind_dict, final_pID_concepts_dict)
    return final_pID_concepts_dict, all_concept_ids, concept_id_to_ind_dict, ind_to_concept_id_dict, person_id_to_ind_dict

def convertConceptsToInds(person_id_to_ind_dict, concept_id_to_ind_dict, pID_concepts_dict):
    # Similar to createMatrix, but just directly do the conversion.
    # dense_mat is a list of lists:
    # dense_mat = [[0, 1, 4], [2, 5, 6], ...] where the internal
    # list contains the index lookup for the concept_id.
    dense_mat = []
    for pID in person_id_to_ind_dict:
        # if pID is not in conds_dict, then indData is an empty array.
        if pID not in pID_concepts_dict:
            dense_mat.append([])
        else:
            # Sort the conds_dict by second element in tuple (datetime)
            currData = sorted(pID_concepts_dict[pID], key = lambda x: x[1])
            # Convert from the full conceptIDs to their indices
            indData  = [concept_id_to_ind_dict[x[0]] for x in currData]
            dense_mat.append(indData)

    # Make sure dense_mat and person_id_to_ind are same input_length
    assert len(dense_mat) == len(person_id_to_ind_dict)
    return dense_mat


In [0]:

# Get the useful rows for a given filename w/o .csv
def queryUsefulRows(filename):
    '''
    Return the useful rows given the OMOP_usefule_columns.csv has already been loaded into pandas.
    Input:
        filename - str, name of input file (e.g. 'person' or 'death'
    Output:
        useful_rows - list, ['col1', 'col2']
    '''
    useful_cols = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/app/OMOP_usefule_columns.csv')
    return list(useful_cols[(useful_cols['TabNam'] == filename) & (useful_cols['Useful'] == True)].ColNam)

# Create a corresponding per-patient lookup dictionary of events from a given OMOP CSV
# For each person_id (abbreviated pID), find all valid:
#     condition_occurrence [condition_concept_id, condition_start_date] # no end_date
#     drug_exposure        [drug_concept_id, drug_exposure_start_date]  # no end_date
#     measurement          [measurement_id, measurement_date] # want value?
#     observation          [observation_concept_id, observation_date] # observation_concept_id or observation_type_concept_id, want value?
#     observation_period   [period_type_concept_id, observation_period_end_date] # leave this out
#     procedure_occurrence [procedure_concept_id, procedure_date]  # vs procedure_type_concept_id
#     visit_occurrence     [visit_concept_id, visit_end_date] # visit_type_concept_id vs visit_concept_id
# that had an occurence. We parse for end date within 1 year of the last visit end date (LVED) in a separate function.
def queryTimelinesFromCSV(fp, column_names):
    '''
        For a given OMOP data CSV from fp, create a lookup table indexed by patient ID that contains
        column_names occurrences.
        Inputs:
            fp - filepath to OMOP CSV
            column_names - a two-element list of column_names to query; element 0 being the concept_id/element 1 being the time.
        Outputs:
            pID_concepts_dict - dictionary where key = person ID, value = zipped list of (concept_id, concept_date)
    '''

    # Split fp by '/' first, then split by period
    filename = fp.split('/')[-1].split('.')[0]

    # Let's get just the indices of the useful rows
    useful_row = queryUsefulRows(filename)

    # let's chunk with 100000 at a time
    # wc -l 34507185
    data_chunker = pd.read_csv(fp, chunksize=100000, usecols=useful_row)

    # person_id to list of tuples, [(condition_concept_id_1, condition_start_date_1),
    #                              (condition_concept_id_2, condition_start_date_2) ...]
    pID_concepts_dict = OrderedDict()

    for chunk in tqdm(data_chunker):
        # Sanity - drop duplicates
        chunk.drop_duplicates(inplace=True)
        chunk.dropna(axis=0, subset=[column_names[1]], inplace=True)

        # Get set of all person_ids in this chunk
        person_ids_chunk = set(chunk.person_id)

        # For each person_id, get one list of (for example) condition_concept_ids,
        # another list of (for example) condition_start_date
        # Look up datafield from input_arg
        # Zip together, and append to the dict
        for person_id in person_ids_chunk:
            # Chunk of dataframe for this person_id
            tmp = chunk[chunk['person_id'] == person_id]
            # Get one list of concept_ids
            concept_id_list = tmp[column_names[0]].tolist()
            # Get one list of dates
            # try:
            date_list = [datetime.strptime(x, '%Y-%m-%d') for x in tmp[column_names[1]].tolist()]
            # except TypeError:
            #    print(tmp[column_names[1]].tolist())
            #    sys.exit(1)

            # Zip together two lists into a list of tuples,
            save_list = list(zip(concept_id_list, date_list))
            # Save into dict
            if person_id in pID_concepts_dict:
                pID_concepts_dict[person_id] = pID_concepts_dict[person_id] + save_list;
            else:
                pID_concepts_dict[person_id] = save_list

            # persons.loc[(persons['person_id'] == person_id),'last_visit_end_date'] = last_visit_end_date

    return pID_concepts_dict

# Count how many possible inds there are, create lookup dicts
def queryAllConcepts(fp, column_name):
    '''
        For a given OMOP data CSV from fp, create a lookup table indexed by patient ID that contains
        column_names occurrences.
        Inputs:
            fp - filepath to OMOP CSV
            column_name - variable to query dataframe by
        Outputs:
            all_concept_ids - dictionary where key = person ID, value = zipped list of (concept_id, concept_date)
            concept_id_to_ind_dict -
            ind_to_concept_id_
    '''
    # Split fp by '/' first, then split by period
    # filename = fp.split('/')[-1].split('.')[0]

    all_concept_ids = set()
    validConcepts = pd.read_csv(fp, usecols=[column_name])
    validConcepts.dropna(axis=0, subset=[column_name], inplace=True)
    all_concept_ids = set(validConcepts[column_name].tolist())

    # Create a lookup for condition_concept_id to ind of numpy array (these are the columns)
    concept_id_to_ind_dict = OrderedDict()
    ind_to_concept_id_dict = OrderedDict()
    for ind, concept_id in enumerate(all_concept_ids):
        concept_id_to_ind_dict[concept_id] = ind
        ind_to_concept_id_dict[ind] = concept_id

    return all_concept_ids, concept_id_to_ind_dict, ind_to_concept_id_dict

def createPersonToInd(train=True):
    if train:
        persons = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/scratch/persons_LVED.csv')
    else:
        persons = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/scratch/persons_LVED_test.csv')

    # We also need a lookup for all person_ids (skip indices potentially)
    person_id_to_ind_dict = OrderedDict()
    ind_to_person_id_dict = OrderedDict()
    for ind, person_id in enumerate(set(persons.person_id.tolist())):
        person_id_to_ind_dict[person_id] = ind
        ind_to_person_id_dict[ind] = person_id

    return person_id_to_ind_dict, ind_to_person_id_dict

def removeEventsWithCutoff(pID_concepts_dict, pID_LVED_dict, concept_id_to_ind_dict, cutoff=6):
    # pID_concepts_dict = copy.deepcopy(in_pID_concepts_dict)
    new_dict = OrderedDict()
    for person_id in tqdm(pID_concepts_dict):
        # Last visit end date - 6 months
        # Something happened with typecasting...
        try:
            lved_cutoff = pID_LVED_dict[person_id] - relativedelta(months=+cutoff)
        except KeyError:
            # person never visited the hospital. delete key.
            # del pID_concepts_dict[person_id]
            continue;

        new_list = []

        for ele in list(pID_concepts_dict[person_id]):
            # If the date of the event is later than 6 months prior to the last_visit_end_date,
            # and occurs in >= 100 patients, keep. Otherwise, continue (pass)
            if ele[1] >= lved_cutoff and ele[0] in concept_id_to_ind_dict:
                new_list.append(ele)
            else:
                continue

        # bad practice, we're modifying a dictionary inside the function
        new_dict[person_id] = new_list

    return new_dict;


In [0]:

#===============================================
# Preprocessing scripts...

preprocess_LVED_deaths(train=True)
make_cond_proc_drug(train=True)


12it [01:41,  8.46s/it]
100%|██████████| 1096/1096 [00:04<00:00, 259.51it/s]
0it [00:00, ?it/s]

Folder for data loaded: /content/drive/My Drive/EHR_Dream_Challenge/train/
Querying timelines...


18it [01:07,  1.06s/it]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:168: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
19it [01:07,  3.57s/it]


Generating all concepts...


  0%|          | 0/63212 [00:00<?, ?it/s]

...

3it [00:49, 16.83s/it]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:176: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
4it [00:49, 11.96s/it]


Generating all concepts...


  0%|          | 0/56982 [00:00<?, ?it/s]

Creating person ID lookup
Using cutoffs to eliminate extraneous data...


100%|██████████| 56982/56982 [00:01<00:00, 47164.62it/s]


In [0]:

#===============================================
#Actual train.py

# leaky ReLU activation
# lrelu = lambda x: tf.keras.layers.LeakyReLU(alpha=0.1)(x)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_dict.pickle', 'rb') as handle:
  conds_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_lookups.pickle', 'rb') as handle:
  conds_lookups = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_dict.pickle', 'rb') as handle:
  procs_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_lookups.pickle', 'rb') as handle:
  procs_lookups = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_dict.pickle', 'rb') as handle:
  drugs_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_lookups.pickle', 'rb') as handle:
  drugs_lookups = pickle.load(handle)

pt_labels = np.load('/content/drive/My Drive/EHR_Dream_Challenge/scratch/Y.npy')
age_days = np.load('/content/drive/My Drive/EHR_Dream_Challenge/scratch/age_days.npy')

# Hyperparameters
PAST_SIZE_CONDS = 125
PAST_SIZE_PROCS = 100
PAST_SIZE_DRUGS = 25
BATCH_SIZE = 256
EPOCHS = 15
embedding_dim = 128
class_weight = {0: 1.,
                1: 10.}

conds_dense = convertConceptsToInds(conds_lookups['person_id_to_ind'], conds_lookups['concept_id_to_ind'], conds_dict)
procs_dense = convertConceptsToInds(procs_lookups['person_id_to_ind'], procs_lookups['concept_id_to_ind'], procs_dict)
drugs_dense = convertConceptsToInds(drugs_lookups['person_id_to_ind'], drugs_lookups['concept_id_to_ind'], drugs_dict)

# this will get changed.
# Set FILL_VAL to be the ind+1 of the dict
FILL_VAL_CONDS = len(conds_lookups['concept_id_to_ind']) + 1
FILL_VAL_PROCS = len(procs_lookups['concept_id_to_ind']) + 1
FILL_VAL_DRUGS = len(drugs_lookups['concept_id_to_ind']) + 1

conds_lookups['ind_to_concept_id'][FILL_VAL_CONDS] = 'MASK'
conds_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_CONDS

procs_lookups['ind_to_concept_id'][FILL_VAL_PROCS] = 'MASK'
procs_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_PROCS

drugs_lookups['ind_to_concept_id'][FILL_VAL_DRUGS] = 'MASK'
drugs_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_DRUGS

# used to be list_of_lists
padded_data_conds = tf.keras.preprocessing.sequence.pad_sequences(
    conds_dense,
    maxlen=PAST_SIZE_CONDS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_CONDS
)

padded_data_procs = tf.keras.preprocessing.sequence.pad_sequences(
    procs_dense,
    maxlen=PAST_SIZE_PROCS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_PROCS
)

padded_data_drugs = tf.keras.preprocessing.sequence.pad_sequences(
    drugs_dense,
    maxlen=PAST_SIZE_DRUGS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_DRUGS
)

train_dataset = tf.data.Dataset.from_tensor_slices((padded_data_conds,
    padded_data_procs, padded_data_drugs, age_days, pt_labels))
train_dataset = train_dataset.batch(BATCH_SIZE)

# leaky ReLU activation
# lrelu = lambda x: tf.keras.layers.LeakyReLU(alpha=0.1)(x)

# conds branch
conds_input = tf.keras.layers.Input(name='condition_list', shape=(PAST_SIZE_CONDS,))
embed_conds = tf.keras.layers.Embedding(FILL_VAL_CONDS + 1, embedding_dim,
    input_length=PAST_SIZE_CONDS)(conds_input)
conds_LSTM  = tf.keras.layers.LSTM(32, activation= 'relu')(embed_conds)

# procs branch
procs_input = tf.keras.layers.Input(name='procedures_list', shape=(PAST_SIZE_PROCS,))
embed_procs = tf.keras.layers.Embedding(FILL_VAL_PROCS + 1, embedding_dim,
    input_length=PAST_SIZE_PROCS)(procs_input)
procs_LSTM  = tf.keras.layers.LSTM(32, activation= 'relu')(embed_procs)

# drugs branch
drugs_input = tf.keras.layers.Input(name='drugs_list', shape=(PAST_SIZE_DRUGS,))
embed_drugs = tf.keras.layers.Embedding(FILL_VAL_DRUGS + 1, embedding_dim,
    input_length=PAST_SIZE_DRUGS)(drugs_input)
drugs_LSTM  = tf.keras.layers.LSTM(32, activation= 'relu')(embed_drugs)

# structured age_days
age_days_input = tf.keras.layers.Input(name='age_in_days', shape=(1,))

# Merge branches
merged = tf.keras.layers.concatenate([conds_LSTM, procs_LSTM, drugs_LSTM, age_days_input])

# Connect with two dense layers
fc1 = tf.keras.layers.Dense(64, activation = 'relu')(merged)
output = tf.keras.layers.Dense(1, activation = 'sigmoid')(fc1)

model = Model(inputs=[conds_input, procs_input, drugs_input, age_days_input], outputs=output)

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])

for epoch in range(EPOCHS):
    print('EPOCH: ' + str(epoch + 1) + '/' + str(EPOCHS))
    for i, (conds_data, procs_data, drugs_data, age_day, pt_label) in tqdm(enumerate(train_dataset)):
        history = model.train_on_batch(x=(conds_data, procs_data, drugs_data, age_day), y=pt_label, class_weight=class_weight)
        print(history)

model.save('/content/drive/My Drive/EHR_Dream_Challenge/model/my_model.h5')

0it [00:00, ?it/s]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
condition_list (InputLayer)     [(None, 125)]        0                                            
__________________________________________________________________________________________________
procedures_list (InputLayer)    [(None, 100)]        0                                            
__________________________________________________________________________________________________
drugs_list (InputLayer)         [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 125, 128)     735232      condition_list[0][0]             
______________________________________________________________________________________________

1it [00:03,  3.81s/it]

[0.0, 1.0]


3it [00:04,  1.97s/it]

...

[0.029362058, 1.0]
[0.16483505, 0.99609375]


381it [01:14,  5.33it/s]

[0.159652, 0.99609375]
[0.15679951, 0.99609375]


383it [01:14,  5.41it/s]

[0.27013066, 0.9921875]
[0.07871001, 1.0]


**This is infer.py, making predictions from the infer(test) data set**

In [0]:
#################
#### INFER ######
#################


# infer preprocessing. -----------------------------

preprocess_LVED_deaths(train=False)
make_cond_proc_drug(train=False)

# real infer ---------------------------------------

PAST_SIZE_CONDS = 125
PAST_SIZE_PROCS = 100
PAST_SIZE_DRUGS = 25

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_dict_test.pickle', 'rb') as handle:
  conds_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/conds_lookups_test.pickle', 'rb') as handle:
  conds_lookups = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_dict_test.pickle', 'rb') as handle:
  procs_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/procs_lookups_test.pickle', 'rb') as handle:
  procs_lookups = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_dict_test.pickle', 'rb') as handle:
  drugs_dict = pickle.load(handle)

with open('/content/drive/My Drive/EHR_Dream_Challenge/scratch/drugs_lookups_test.pickle', 'rb') as handle:
  drugs_lookups = pickle.load(handle)

age_days = np.load('/content/drive/My Drive/EHR_Dream_Challenge/scratch/age_days_test.npy')

# conds_mat_test = createMatrix(conds_lookups['person_id_to_ind'], conds_lookups['concept_id_to_ind'], conds_dict)
assert conds_lookups['person_id_to_ind'] == procs_lookups['person_id_to_ind']
assert conds_lookups['person_id_to_ind'] == drugs_lookups['person_id_to_ind']

conds_dense = convertConceptsToInds(conds_lookups['person_id_to_ind'], conds_lookups['concept_id_to_ind'], conds_dict)
procs_dense = convertConceptsToInds(procs_lookups['person_id_to_ind'], procs_lookups['concept_id_to_ind'], procs_dict)
drugs_dense = convertConceptsToInds(drugs_lookups['person_id_to_ind'], drugs_lookups['concept_id_to_ind'], drugs_dict)

model = tf.keras.models.load_model('/content/drive/My Drive/EHR_Dream_Challenge/model/my_model.h5')

#cond_dataset_test = tf.data.Dataset.from_tensor_slices((conds_mat_test))

model.summary()

# this will get changed.
# Set FILL_VAL to be the ind+1 of the dict
FILL_VAL_CONDS = len(conds_lookups['concept_id_to_ind']) + 1
FILL_VAL_PROCS = len(procs_lookups['concept_id_to_ind']) + 1
FILL_VAL_DRUGS = len(drugs_lookups['concept_id_to_ind']) + 1

conds_lookups['ind_to_concept_id'][FILL_VAL_CONDS] = 'MASK'
conds_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_CONDS

procs_lookups['ind_to_concept_id'][FILL_VAL_PROCS] = 'MASK'
procs_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_PROCS

drugs_lookups['ind_to_concept_id'][FILL_VAL_DRUGS] = 'MASK'
drugs_lookups['concept_id_to_ind']['MASK'] = FILL_VAL_DRUGS

# used to be list_of_lists
padded_data_conds = tf.keras.preprocessing.sequence.pad_sequences(
    conds_dense,
    maxlen=PAST_SIZE_CONDS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_CONDS
)

padded_data_procs = tf.keras.preprocessing.sequence.pad_sequences(
    procs_dense,
    maxlen=PAST_SIZE_PROCS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_PROCS
)

padded_data_drugs = tf.keras.preprocessing.sequence.pad_sequences(
    drugs_dense,
    maxlen=PAST_SIZE_DRUGS,
    dtype='int32',
    padding='post',
    truncating='pre',
    value= FILL_VAL_DRUGS
)

predictions=model.predict((padded_data_conds, padded_data_procs, padded_data_drugs, age_days.astype(np.float32)))

match_output = OrderedDict()
for i, pID in enumerate(conds_lookups['person_id_to_ind']):
    match_output[pID] = predictions[i]

persons = pd.read_csv('/content/drive/My Drive/EHR_Dream_Challenge/infer/person.csv', usecols=['person_id'])

final_out = []
for i in range(len(persons)):
    final_out.append(match_output[persons.iloc[i].person_id][0])

persons['score'] = final_out
persons.to_csv('/content/drive/My Drive/EHR_Dream_Challenge/output/predictions.csv')


9it [00:33,  2.15s/it]


Folder for data loaded: /content/drive/My Drive/EHR_Dream_Challenge/infer/
Querying timelines...


16it [00:33,  1.02it/s]
 22%|██▏       | 4980/22764 [00:00<00:00, 49793.87it/s]

Generating all concepts...
Creating person ID lookup
Using cutoffs to eliminate extraneous data...


100%|██████████| 22764/22764 [00:00<00:00, 38435.09it/s]


...

Querying timelines...


2it [00:19,  9.94s/it]
 26%|██▌       | 5627/21727 [00:00<00:00, 56265.36it/s]

Generating all concepts...
Creating person ID lookup
Using cutoffs to eliminate extraneous data...


100%|██████████| 21727/21727 [00:00<00:00, 59934.43it/s]


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
condition_list (InputLayer)     [(None, 125)]        0                                            
__________________________________________________________________________________________________
procedures_list (InputLayer)    [(None, 100)]        0                                            
__________________________________________________________________________________________________
drugs_list (InputLayer)         [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 125, 128)     735232      condition_list[0][0]             
______________________________________________________________________________________________

In [0]:
persons.head(200)

,person_id,score
0,9879,0.000000
1,640,0.005465
2,35370,0.159226
3,11594,0.095814
4,24438,0.102249
...,...,...
195,35852,0.077204
196,3851,0.084079
197,3621,0.078149
198,37003,0.088132
